In [5]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import models, transforms, datasets

import torch.optim as optim
from torch.utils.data import DataLoader

import PIL.Image as Image
from tqdm import tqdm
import os
import time

from sklearn.metrics import f1_score

from model.vgg_16 import *
from model.hybrid_CNN import *


In [6]:
# hyper parameters
num_epochs = 1
num_classes = 2
batch_size = 1 # WE WANT IMAGES TO PASS HYBRID CONV LAYER ONE BY ONE
learning_rate = 0.001
model_name = vgg16(pretrained=True)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
# get dataset
def load_data(batch_size):
    """
    return the train/val/test dataloader
    """
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
    ])
    
    train_dataset = datasets.CelebA(root='./data',
                                    split='train',
                                    target_type='attr',
                                    transform=transform,
                                    download=False)
    val_dataset = datasets.CelebA(root='./data',
                                    split='valid',
                                    target_type='attr',
                                    transform=transform,
                                    download=False)
    test_dataset = datasets.CelebA(root='./data',
                                    split='test',
                                    target_type='attr',
                                    transform=transform,
                                    download=False)

    # data loader
    train_loader = DataLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                shuffle=True)
    val_loader = DataLoader(dataset=val_dataset,
                                batch_size=batch_size,
                                shuffle=False)
    test_loader = DataLoader(dataset=test_dataset,
                                batch_size=batch_size,
                                shuffle=False)
    
    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data(batch_size)

In [8]:
def initialize_model(model_name, learning_rate, num_classes, device):
    """
    initialize the model (pretrained vgg16_bn)
    define loss function and optimizer and move data to gpu if available
    
    return:
        model, loss function(criterion), optimizer
    """
    # model = models.vgg16_bn(pretrained=True)                ############## add param here? ################
    # num_ftrs = model.classifier[6].in_features
    # model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    
    model = model_name.to(device)
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    return model, criterion, optimizer


model, criterion, optimizer = initialize_model(model_name, learning_rate, num_classes, device)

In [9]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [17]:
layers = list(model.features.children())[:-1]
# layers[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
# features = nn.Sequential(*layers).cuda()
list(model.features.children())
layers

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(512, 512, kernel_size=(3, 3), stride=(1

In [5]:
def make_plots(step_hist, loss_hist, epoch=0):
    plt.plot(step_hist, loss_hist)
    plt.xlabel('train_iterations')
    plt.ylabel('Loss')
    plt.title('epoch'+str(epoch+1))
    plt.savefig('epoch_1')
    plt.clf()

In [6]:
def train(train_loader, model, criterion, optimizer, num_epochs, device):
    """
    Move data to GPU memory and train for specified number of epochs
    Also plot the loss function and save it in `Figures/`
    Trained model is saved as `cnn.ckpt`
    """
    for epoch in range(num_epochs): # repeat the entire training `num_epochs` times
        # for each training sample
        loss_hist = []
        step_hist = []
        for i, (images, labels) in tqdm(enumerate(train_loader)):
            # move to gpu if available
            label = labels[:, 2]   # attractiveness label
            cov_attr = labels[:, 20]    # gender (male/female)   
            cov_attr = (cov_attr + 1) // 2  # map from {-1, 1} to {0, 1}
            
            images = images.to(device)
            label = label.to(device)
            
            # forward pass
            outputs = model(images, cov_attr)    # model takes covariate here
            loss = criterion(outputs, label) 
            
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # print
            if (i+1) % 100 == 0:
                print('Epoch: [{}/{}], Step[{}/{}], Loss:{:.4f}' \
                    .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
                loss_hist.append(loss.item())
                step_hist.append(i+1)
        
        make_plots(step_hist, loss_hist, epoch)
        
    torch.save(model.state_dict(), 'cnn2.ckpt')

In [8]:
# train model
train(train_loader, model, criterion, optimizer, num_epochs, device)

1it [00:01,  1.26s/it]

outputs:  tensor([[-0.0158,  0.0180]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([0], device='cuda:0')
outputs:  tensor([[ 21.9504, -18.9808]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([0], device='cuda:0')


5it [00:01,  1.54it/s]

outputs:  tensor([[ 24.5200, -21.3002]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([0], device='cuda:0')
outputs:  tensor([[ 32.7455, -28.5372]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([1], device='cuda:0')
outputs:  tensor([[ 27.6774, -25.3791]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([0], device='cuda:0')
outputs: 

7it [00:01,  2.11it/s]

...

185it [00:13, 15.15it/s]

 tensor([[-0.0316,  0.0177]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([1], device='cuda:0')
outputs:  tensor([[-0.0680, -0.1661]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([1], device='cuda:0')
outputs:  tensor([[-0.0266, -0.0173]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([0], device='cuda:0')


186it [00:13, 13.62it/s]


outputs:  tensor([[-0.0539, -0.1965]], device='cuda:0', grad_fn=<AddmmBackward>)
label:  tensor([0], device='cuda:0')


KeyboardInterrupt: 

In [ ]:
def evaluate(val_loader, model, device):
    """
    Run the validation set on the trained model
    """
    model.eval() # BatchNorm uses moving mean/variance instead of mini-batch mean/variance
    with torch.no_grad():
        # initialize the stats
        correct = 0
        total = 0
        # pass through testing data once
        for images, labels in val_loader:

            label = labels[:, 2]
            # again move to device first
            images = images.to(device)
            label = label.to(device)
            # forward once
            outputs = model(images)
            # instead of calculating loss we will get predictions
            # it's essetially outputs just reformatting imo
            _, predicted = torch.max(outputs.data, 1)
            # accumulate stats
            total += label.size(0) # yeah again, number of elements in the tensor
            correct += (label == predicted).sum().item()

        # print
        print('Test accuracy on 10000 test images: {}%' \
                .format(100 * correct / total))
        
# run validation set
evaluate(val_loader, model, device)